In [8]:
!pip install pandas gradio catboost scikit-learn openpyxl



In [ ]:
import gradio as gr
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder


df = pd.read_excel("Football Data Test Task-DESKTOP-HKE84D6.xlsx", sheet_name="Raw Data")
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date').reset_index(drop=True)

stat_map = {
    'Goals': ('FTHG', 'FTAG'),
    'Shots': ('HS', 'AS'),
    'ShotsOnTarget': ('HST', 'AST'),
    'Fouls': ('HF', 'AF'),
    'Corners': ('HC', 'AC'),
}
windows = [5, 15, 38]
team_stats = defaultdict(lambda: defaultdict(list))
new_features = []

for idx, row in df.iterrows():
    home, away = row['HomeTeam'], row['AwayTeam']
    for stat_name, (home_col, away_col) in stat_map.items():
        for window in windows:
            h_feat, a_feat = f"{stat_name}_Home_Last{window}", f"{stat_name}_Away_Last{window}"
            if h_feat not in new_features: new_features.append(h_feat)
            if a_feat not in new_features: new_features.append(a_feat)
            df.loc[idx, h_feat] = sum(team_stats[home][stat_name][-window:]) if team_stats[home][stat_name] else 0
            df.loc[idx, a_feat] = sum(team_stats[away][stat_name][-window:]) if team_stats[away][stat_name] else 0
    for stat_name, (home_col, away_col) in stat_map.items():
        team_stats[home][stat_name].append(row[home_col])
        team_stats[away][stat_name].append(row[away_col])


def get_result(row):
    if row['FTHG'] > row['FTAG']: return 'H'
    elif row['FTHG'] < row['FTAG']: return 'A'
    return 'D'
df['Result'] = df.apply(get_result, axis=1)

df = df.dropna(subset=new_features)


X = df[new_features]
y = df['Result']
le = LabelEncoder()
y_encoded = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, shuffle=False)

model = CatBoostClassifier(verbose=0)
model.fit(X_train, y_train)


def build_features_for_match(home, away, date, data):
    date = pd.to_datetime(date)
    past_df = data[data['Date'] < date]
    team_stats_hist = defaultdict(lambda: defaultdict(list))

    for _, row in past_df.iterrows():
        h, a = row['HomeTeam'], row['AwayTeam']
        for stat_name, (home_col, away_col) in stat_map.items():
            team_stats_hist[h][stat_name].append(row[home_col])
            team_stats_hist[a][stat_name].append(row[away_col])

    features = {}
    for stat_name in stat_map.keys():
        for window in windows:
            h_key, a_key = f"{stat_name}_Home_Last{window}", f"{stat_name}_Away_Last{window}"
            features[h_key] = sum(team_stats_hist[home][stat_name][-window:]) if team_stats_hist[home][stat_name] else 0
            features[a_key] = sum(team_stats_hist[away][stat_name][-window:]) if team_stats_hist[away][stat_name] else 0
    return pd.DataFrame([features])


def predict_match(home_team, away_team, match_date):
    input_df = build_features_for_match(home_team, away_team, match_date, df)
    if input_df.isnull().values.any():
        return "❌ Not enough historical data to predict this match."

    probs = model.predict_proba(input_df)[0]
    class_map = dict(zip(le.classes_, probs))

    result = f"📊 Prediction for {home_team} vs {away_team} on {match_date}:\n\n"
    result += f"🏠 Home Team Winning % (H): {class_map.get('H', 0)*100:.2f}%\n"
    result += f"🤝 Draw Possibility % (D): {class_map.get('D', 0)*100:.2f}%\n"
    result += f"🛫 Away Team Winning % (A): {class_map.get('A', 0)*100:.2f}%\n"
    return result


interface = gr.Interface(
    fn=predict_match,
    inputs=[
        gr.Textbox(label="Home Team"),
        gr.Textbox(label="Away Team"),
        gr.Textbox(label="Match Date (YYYY-MM-DD)")
    ],
    outputs=gr.Textbox(label="Prediction Result"),
    title="⚽ Football Match Outcome Predictor",
    description="Enter the teams and match date to see the predicted winning percentages.\n(H = Home Win, D = Draw, A = Away Win)"
)


interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8c81c5ee032c0bbe0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
